# Mattes et al. (2022) data preparation

Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from scipy.stats import ttest_ind

## Read data from Stahl et al.

In [ ]:
data_spe = pd.read_csv('../data/mattes_dataset/spe_dataset.csv')
data_spe.head()

Standardize EEG data and create y for DDM

In [ ]:
def standardize(series):
    return series  / series.std()

data_spe['pre_Ne_standarized'] = data_spe.groupby('subj_idx')['pre_Ne'].transform(standardize)
data_spe['acc'] = [-1 if response == 0 else 1 for response in data_spe['response']]
data_spe['pre_response'] = [-1 if type == 'error' else 1 for type in data_spe['type']]
data_spe['y'] = data_spe['rt'] * data_spe['acc']

In [ ]:
data_spe.head()

Prepare json file for Stan

In [ ]:
y = data_spe['y'].to_list()
pre_acc = data_spe['pre_response'].to_list()
pre_ne = data_spe['pre_Ne_standarized'].to_list()
unique_ids = data_spe['subj_idx'].unique()
id_mapping = {original_id: new_id for new_id, original_id in enumerate(unique_ids, start=1)}
data_spe['participant_idx'] = data_spe['subj_idx'].map(id_mapping)

participant_index = data_spe['participant_idx'].to_list()

n_participants = len(np.unique(participant_index))
n_conditions = 1
condition = list(np.ones((len(participant_index),)))
participants_trials_slices = []
for index in np.unique(participant_index):
    indices = np.where(participant_index == index)[0]
    start_index = int(indices[0] + 1)
    end_index = int(indices[-1] + 1)
    participants_trials_slices.append([start_index, end_index])

In [ ]:
data_2d = {
    "N": len(y),
    "participants_trials_slices": participants_trials_slices,
    "n_conditions": n_conditions,
    "n_participants": n_participants,
    "y": y,
    "condition": condition,
    'pre_ne': pre_ne,
    'pre_acc': pre_acc,
    "participant": participant_index
}

Save data to json

In [ ]:
with open("../data/mattes_dataset/stahl_spe_data_standardized.json", "w") as outfile: 
    json.dump(data_2d, outfile)

## Analyse data

### 1. Post-error adaptation in RTs

Group level

In [ ]:
sns.histplot(
    data_spe,
    x='rt',
    hue='pre_response',
)

Participant-level

In [ ]:
g = sns.FacetGrid(
    data_spe.sort_values(['subj_idx']),
    col="subj_idx",
    col_wrap=3,
    sharex=False,
    sharey=False,
    aspect=2,
)

g.map_dataframe(
    sns.histplot,
    x="rt",
    hue='pre_response',
    kde=True,
    palette='colorblind'
)

In [ ]:
print(f"Mean post error RT: {np.mean(data_spe[data_spe['pre_response'] == -1]['rt'])}")
print(f"Mean post correct RT: {np.mean(data_spe[data_spe['pre_response'] == 1]['rt'])}")

print(f"post error vs post correct : {ttest_ind(data_spe[data_spe['pre_response'] == -1]['rt'], data_spe[data_spe['pre_response'] == 1]['rt'])}")

#### Test post-response adaptation and links with pre-trial accuracy and brain signal

In [ ]:
import statsmodels.formula.api as smf

mod = smf.ols(formula='rt ~ pre_Ne_standarized*pre_response', data=data_spe)
res = mod.fit()

print(res.summary())